# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [8]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterMessages', con = engine)
X = df['message']
y = df.drop(['message','original','genre','id'], axis = 1)
X.shape, y.shape

((26215,), (26215, 35))

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f3a42565c80>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
def perf_report(model, X_test, y_test):
    '''
    Function to generate classification report on the model
    Input: Model, test set ie X_test & y_test
    Output: Prints the Classification report
    '''
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [15]:
perf_report(pipeline, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.62      0.36      0.45      1240
          1       0.82      0.93      0.87      4003

avg / total       0.77      0.80      0.77      5243

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      4322
          1       0.83      0.38      0.52       921

avg / total       0.87      0.88      0.86      5243

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      5243

aid_related
             precision    recall  f1-score   support

          0       0.71      0.88      0.79      3063
          1       0.76      0.50      0.60      2180

avg / total       0.73      0.73      0.71      5243

medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
parameters =  {'tfidf__use_idf': (True, False),
               #'clf__estimator__max_depth': [2, 5, None],
               'clf__estimator__criterion': ['entropy'],
               'clf__estimator__n_estimators':[50, 100]} 

cv = GridSearchCV(pipeline, param_grid=parameters, cv = 2, n_jobs=-1, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
cv.fit(X_train, y_train)
perf_report(cv, X_test, y_test)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] clf__estimator__criterion=entropy, clf__estimator__n_estimators=50, tfidf__use_idf=True 
[CV]  clf__estimator__criterion=entropy, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.23002098035475874, total= 2.3min
[CV] clf__estimator__criterion=entropy, clf__estimator__n_estimators=50, tfidf__use_idf=True 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV]  clf__estimator__criterion=entropy, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.2337402250619874, total= 2.3min
[CV] clf__estimator__criterion=entropy, clf__estimator__n_estimators=50, tfidf__use_idf=False 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  5.0min remaining:    0.0s


[CV]  clf__estimator__criterion=entropy, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.22668319664314324, total= 2.2min
[CV] clf__estimator__criterion=entropy, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__criterion=entropy, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.22582490940301353, total= 2.2min
[CV] clf__estimator__criterion=entropy, clf__estimator__n_estimators=100, tfidf__use_idf=True 
[CV]  clf__estimator__criterion=entropy, clf__estimator__n_estimators=100, tfidf__use_idf=True, score=0.2350753385466336, total= 4.4min
[CV] clf__estimator__criterion=entropy, clf__estimator__n_estimators=100, tfidf__use_idf=True 
[CV]  clf__estimator__criterion=entropy, clf__estimator__n_estimators=100, tfidf__use_idf=True, score=0.22944878886133893, total= 4.4min
[CV] clf__estimator__criterion=entropy, clf__estimator__n_estimators=100, tfidf__use_idf=False 
[CV]  clf__estimator__criterion=entropy, clf__estimator__n_estimators=10

[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 28.7min finished


related
             precision    recall  f1-score   support

          0       0.74      0.28      0.40      1240
          1       0.81      0.97      0.88      4003

avg / total       0.80      0.81      0.77      5243

request
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      4322
          1       0.89      0.41      0.56       921

avg / total       0.89      0.89      0.87      5243

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      5243

aid_related
             precision    recall  f1-score   support

          0       0.76      0.90      0.82      3063
          1       0.81      0.60      0.69      2180

avg / total       0.78      0.77      0.77      5243

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
cv.best_params_

{'clf__estimator__criterion': 'entropy',
 'clf__estimator__n_estimators': 100,
 'tfidf__use_idf': True}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
from sklearn.svm import SVC

pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',  MultiOutputClassifier(SVC()))
])

pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f3a42565c80>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
     max_iter=-1, probability=False, random_state=None, shrinking=True,
     tol=0.001, verbose=False),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8

In [18]:
parameters2 = {#'clf__estimator__kernel': ['rbf','linear'], 
                'clf__estimator__gamma': [1e-3, 1e-4],
                'clf__estimator__C': [1, 10, 100, 1000]}


cv2 = GridSearchCV(pipeline2, param_grid = parameters2, cv = 2, n_jobs=-1, verbose=3)
cv2.fit(X_train, y_train)
perf_report(cv2, X_test, y_test)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] clf__estimator__C=1, clf__estimator__gamma=0.001 ................
[CV]  clf__estimator__C=1, clf__estimator__gamma=0.001, score=0.20589357238222392, total= 6.0min
[CV] clf__estimator__C=1, clf__estimator__gamma=0.001 ................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  8.9min remaining:    0.0s


[CV]  clf__estimator__C=1, clf__estimator__gamma=0.001, score=0.19998092695021935, total= 6.0min
[CV] clf__estimator__C=1, clf__estimator__gamma=0.0001 ...............


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 17.7min remaining:    0.0s


[CV]  clf__estimator__C=1, clf__estimator__gamma=0.0001, score=0.20589357238222392, total= 5.3min
[CV] clf__estimator__C=1, clf__estimator__gamma=0.0001 ...............
[CV]  clf__estimator__C=1, clf__estimator__gamma=0.0001, score=0.19998092695021935, total= 5.3min
[CV] clf__estimator__C=10, clf__estimator__gamma=0.001 ...............
[CV]  clf__estimator__C=10, clf__estimator__gamma=0.001, score=0.20341407591073812, total= 7.9min
[CV] clf__estimator__C=10, clf__estimator__gamma=0.001 ...............
[CV]  clf__estimator__C=10, clf__estimator__gamma=0.001, score=0.19816898722105664, total= 8.0min
[CV] clf__estimator__C=10, clf__estimator__gamma=0.0001 ..............
[CV]  clf__estimator__C=10, clf__estimator__gamma=0.0001, score=0.20589357238222392, total= 6.0min
[CV] clf__estimator__C=10, clf__estimator__gamma=0.0001 ..............
[CV]  clf__estimator__C=10, clf__estimator__gamma=0.0001, score=0.19998092695021935, total= 6.0min
[CV] clf__estimator__C=100, clf__estimator__gamma=0.001

[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 169.5min finished


related
             precision    recall  f1-score   support

          0       0.66      0.56      0.61      1240
          1       0.87      0.91      0.89      4003

avg / total       0.82      0.83      0.82      5243

request
             precision    recall  f1-score   support

          0       0.92      0.97      0.94      4322
          1       0.80      0.61      0.69       921

avg / total       0.90      0.90      0.90      5243

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      5243

aid_related
             precision    recall  f1-score   support

          0       0.80      0.82      0.81      3063
          1       0.74      0.71      0.72      2180

avg / total       0.77      0.78      0.77      5243

medical_help
             precision    recall  f1-score   support

          0       0.94      0.99      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [19]:
cv2.best_params_

{'clf__estimator__C': 1000, 'clf__estimator__gamma': 0.001}

### 9. Export your model as a pickle file

In [ ]:
# save the model to disk
filename = 'model.pkl'
pickle.dump(cv2, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.